# Assignment 1
Course: Mathematics of Generative Models \
Name: Md Faizal Karim \
Roll No: 2022121004

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
import queue
import graphviz

plt.style.use('seaborn-v0_8')           
np.__version__

'1.24.4'

## Problem 1

<img src= './soln1.jpeg' width='60%'>
Hence, we obtain p = 2/3

#### Simulating a Bernoulli random variable (p = 2/3) using numpy

In [2]:
x = np.random.binomial(n = 1, p = 2/3, size=30)
x = ['G' if i == 1 else 'B' for i in x]

print(x)

['G', 'G', 'B', 'G', 'G', 'B', 'G', 'B', 'B', 'G', 'G', 'G', 'B', 'B', 'G', 'G', 'B', 'G', 'B', 'B', 'B', 'B', 'G', 'G', 'G', 'G', 'G', 'G', 'B', 'B']


## Problem 2

In [5]:
def generate(n, samples, plot_cdf=False):
    """
    Input:
        n: n-sided die,
        samples: outcome of k trials
        plot_cdf: plot the cdf?
    Returns:
        out: a random sample from the calculated distribution
    """
    
    pmf = np.histogram(samples, bins=n)[0]/len(samples)
    cdf = np.cumsum(pmf)

    if plot_cdf:
        plt.title('CDF plot')
        plt.plot(cdf, label='cdf')
        plt.legend()
        plt.show()

    u = np.random.uniform(0, 1, 1)
    out = np.abs(cdf - u)
    out = np.where(np.abs(out - np.min(out)) < 1e-5)[0] + 1

    return out[np.random.choice(1, len(out))] if len(out) != 1 else out
    

In [6]:
samples = [1, 3, 2, 4, 2, 3, 5, 7, 3, 2]

for i in range(10):
    print(generate(n=6, samples=samples))

[3]
[2]
[1]
[2]
[3]
[5]
[6]
[1]
[4]
[2]


## Problem 3

In [14]:
samples = np.array([
    [1, 0, 1, 0, 0, 1, 1, 0, 0, 1],
    [0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
    [1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
    [1, 0, 1, 0, 1, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 1, 0, 1, 1],
    [1, 1, 0, 0, 1, 1, 0, 1, 1, 1],
    [0, 1, 0, 0, 1, 0, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 0, 1, 0]])

class Node:
    def __init__(self, prob=None, index=0, is_leaf=False):
        self.prob = prob
        self.is_leaf = is_leaf
        self.left = None
        self.right = None
        self.index = index
        
    def __repr__(self):
        return f"node: {id(self)}, prob:{self.prob:.4f}, index:{self.index}"
        

In [99]:

class Solver:
    def __init__(self, samples):
        self.samples = samples
        self.n_samples = len(samples[0])
        self.root = None
        
    def start(self, node):
        left_prob = len(np.where(self.samples[:, 0] == 1)[0])/len(self.samples)
        right_prob = 1 - left_prob
        node.left = Node(prob=left_prob, index=1)
        node.right = Node(prob=right_prob, index=1)
        return node.left, node.right
    
    def fxn(self, node, i, condition):
        left_prob = len(np.where((self.samples[:, i] == 1) & (self.samples[:, i-1] == condition))[0])/len(np.where(self.samples[:, i-1] == condition)[0])
        right_prob = 1 - left_prob
        node.left = Node(prob=left_prob, index=i)
        node.right = Node(prob=right_prob, index=i)
        return node.left, node.right
    
    def solve(self, ):
        q = queue.Queue()
        self.root = Node()
        left, right = self.start(self.root)
        q.put(left)
        q.put(right)
                
        while q.qsize() != 0:
            for condition in [0, 1]:
                temp_node = q.get()
                if temp_node.index == self.n_samples - 1:
                    continue
                
                if temp_node is None:
                    continue
                
                left, right = self.fxn(temp_node, temp_node.index + 1, condition)
                q.put(left)
                q.put(right)
                
                
        print("Done")
        return self.root
        

In [100]:
solver = Solver(samples=samples)
root = solver.solve()

Done


In [101]:
def create_graph(node, graph=None):
    if graph is None:
        graph = graphviz.Digraph(format='png', graph_attr={'rankdir': 'LR'})

    if node is not None:
        graph.node(str(id(node)), 
                   label=f"Prob: {node.prob:.4f}" if node.prob is not None else 'Start',
                   style='filled',
                   fillcolor='lightblue' if node.prob is None else 'white')

        if node.left is not None:
            graph.edge(str(id(node)), str(id(node.left)), label='1')
            create_graph(node.left, graph)

        if node.right is not None:
            graph.edge(str(id(node)), str(id(node.right)), label='0')
            create_graph(node.right, graph)

    return graph


graph = create_graph(root)


In [103]:
graph.render("tree_graph", cleanup=True)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.889175 to fit


'tree_graph.png'

In [104]:
def generate(root):
    result = []
    
    def fxn(node):
        if node is None or node.left is None or node.right is None:
            return 
        out = np.random.binomial(1, node.left.prob, 1)[0]
        result.append(out)
        fxn(node.left if out == 1 else node.right)
    
    fxn(root)
    return result            

In [138]:
generated_samples = []

for i in range(100):
    generated_samples.append(generate(root))
    
generated_samples[:10]

[[0, 1, 0, 0, 1, 1, 0, 0, 1],
 [0, 1, 1, 0, 0, 1, 1, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 1, 1],
 [0, 0, 1, 1, 1, 1, 1, 1, 0],
 [1, 0, 0, 1, 1, 1, 0, 0, 1],
 [1, 1, 1, 0, 1, 1, 0, 1, 0],
 [1, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 0, 0, 1, 0, 1]]

In [139]:
def find_intelligen_human(samples):
    def has_101(arr):
        for i in range(0, len(arr) - 2):
            if [arr[i], arr[i+1], arr[i+2]] == [1, 0, 1]:
                return True
            
        return False
    
    def has_11(arr):
        for i in range(0, len(arr) - 1):
            if [arr[i], arr[i + 1]] == [1, 1]:
                return True
            
        return False
    
    counter = 0
    for sample in samples:
        counter += has_101(sample) & has_11(sample)
        
    print(f"{counter} intelligent out of {len(samples)}")

In [140]:
find_intelligen_human(generated_samples)

36 intelligent out of 100
